<a href="https://colab.research.google.com/github//pylabel-project/samples/blob/main/yolov5_training.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> 

## Simple YoloV5 Training Example
This notebook demonstrates how to use your own labeled dataset to train a YoloV5 model and then use it to make predictions. For this demonstration we will download 100 images from the <a href="https://github.com/pylabel-project/datasets_models#squirrels-and-nuts">squirrels and nuts dataset.</a> (Which I created from images of squirrels in my yard.)


In [1]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)
import os 
import zipfile

## Download Images and Labels

In [2]:
os.makedirs("data/", exist_ok=True)
!wget "https://github.com/pylabel-project/datasets_models/blob/main/squirrelsandnuts/squirrelsandnuts_train.zip?raw=true" -O data/squirrelsandnuts_train.zip
with zipfile.ZipFile("data/squirrelsandnuts_train.zip", 'r') as zip_ref:
    zip_ref.extractall("data/")

--2021-12-07 04:42:36--  https://github.com/pylabel-project/datasets_models/blob/main/squirrelsandnuts/squirrelsandnuts_train.zip?raw=true
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/pylabel-project/datasets_models/raw/main/squirrelsandnuts/squirrelsandnuts_train.zip [following]
--2021-12-07 04:42:36--  https://github.com/pylabel-project/datasets_models/raw/main/squirrelsandnuts/squirrelsandnuts_train.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pylabel-project/datasets_models/main/squirrelsandnuts/squirrelsandnuts_train.zip [following]
--2021-12-07 04:42:37--  https://raw.githubusercontent.com/pylabel-project/datasets_models/main/squirrelsandnuts/squirrelsandnuts_train.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com

Now you should have 100 images and annotation files in the folder data/squirrelsandnuts_train/. There is also a YAML file that you need to tell YOLO where to find the images and folders. View the Yaml file. Note that there are 2 classes: squirrels and nuts. 

In [3]:
!cat data/squirrelsandnuts_train/yolo_data.yaml

path: ../data/squirrelsandnuts_train/
train: images/train  # train images (relative to 'path') 128 images
val: images/train  # val images (relative to 'path') 128 images

# Classes
nc: 2  # number of classes
names: [ 'Squirrel','Nut']  # class names

# Install Yolo Dependencies
This is known to work on Google Colab, it may or may not work in other environments. 

In [4]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install
import torch
from yolov5 import utils

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


Now you are ready to train the model. Below is the standard code that is used for training which I customized for this particulat dataset. Some of the key parameters:   
- **img** This should be the longest dimension of your images in pixels. The squirrel photos are 960 pixels wide.
- **data** The path to the yaml file, from the yolov5 directory. 
- **weights** Start from the standard yolo pretraining model. 
- **epochs** It is set to 10 so it goes faster while you are still testing things. I would suggest doing at least 100 epochs for your actual model training. 

In [5]:
!python train.py --img 960 --batch 16 --epochs 10 --data ../data/squirrelsandnuts_train/yolo_data.yaml --weights yolov5s.pt --cache --exist-ok


train: weights=yolov5s.pt, cfg=, data=../data/squirrelsandnuts_train/yolo_data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=10, batch_size=16, imgsz=960, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=True, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-124-g1075488 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0

After the model has completed training: 
- Review the model summary above. Check the precision, recall, and mAP values for all of the classes. If there is a class that has worse values than the others, annotate more samples for that class. 
- The model weights will be saved to **runs/train/exp/weights/best.pt**. 

Now you can use that model to make predictions.

In [6]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/exp/weights/best.pt', force_reload=True) 
imgs = ['../data/squirrelsandnuts_train/images/train/2021-07-03T06-30-10-frame_0001.jpeg']  # batch of images
results = model(imgs)

results.save()
results.pandas().xyxy[0]


Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2021-12-7 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
Saved 1 image to runs/detect/exp4


,xmin,ymin,xmax,ymax,confidence,class,name


Look at the output above. Look in the runs/detect folder. If there are no classes detected then the model is not working well and you need more data, more epochs, or to try other things. 